In [1]:
import json
import pandas as pd

In [2]:
with open("../data/raw/events.json", "r") as f:
    events = json.load(f)

df = pd.json_normalize(events)
df

,type,userId,anonymousId,timestamp,traits.email,traits.plan,context.device.type,event,properties.method,properties.feature_name
0,identify,user_001,anon_123,2024-01-01T09:00:00Z,user1@example.com,free,mobile,NaN,NaN,NaN
1,track,user_001,NaN,2024-01-01T09:02:00Z,NaN,NaN,NaN,Signup Completed,email,NaN
2,track,user_001,NaN,2024-01-02T08:30:00Z,NaN,NaN,NaN,Login,NaN,NaN
3,track,user_001,NaN,2024-01-02T08:35:00Z,NaN,NaN,NaN,Feature Used,NaN,dashboard
4,track,NaN,anon_456,2024-01-02T10:00:00Z,NaN,NaN,NaN,Feature Used,NaN,pricing_page
5,identify,user_002,anon_456,2024-01-02T10:05:00Z,user2@example.com,pro,NaN,NaN,NaN,NaN


### Convert Timestamp to Datetime

In [3]:
df["timestamp"] = pd.to_datetime(df["timestamp"])
df.dtypes

type                                    object
userId                                  object
anonymousId                             object
timestamp                  datetime64[ns, UTC]
traits.email                            object
traits.plan                             object
context.device.type                     object
event                                   object
properties.method                       object
properties.feature_name                 object
dtype: object

### Sort Events by Time

In [4]:
df_sorted = df.sort_values("timestamp")
df_sorted[["type", "event", "userId", "anonymousId", "timestamp"]]

,type,event,userId,anonymousId,timestamp
0,identify,NaN,user_001,anon_123,2024-01-01 09:00:00+00:00
1,track,Signup Completed,user_001,NaN,2024-01-01 09:02:00+00:00
2,track,Login,user_001,NaN,2024-01-02 08:30:00+00:00
3,track,Feature Used,user_001,NaN,2024-01-02 08:35:00+00:00
4,track,Feature Used,NaN,anon_456,2024-01-02 10:00:00+00:00
5,identify,NaN,user_002,anon_456,2024-01-02 10:05:00+00:00


### Count Users

In [5]:
df_sorted["userId"].dropna().unique()

array(['user_001', 'user_002'], dtype=object)

In [6]:
df_sorted["userId"].nunique()

2

In [7]:
df_sorted["anonymousId"].dropna().unique()

array(['anon_123', 'anon_456'], dtype=object)

### Create a “Resolved User ID

In [8]:
df_sorted["resolved_user"] = df_sorted["userId"].fillna(df_sorted["anonymousId"])
df_sorted[["type", "event", "userId", "anonymousId", "resolved_user", "timestamp"]]

,type,event,userId,anonymousId,resolved_user,timestamp
0,identify,NaN,user_001,anon_123,user_001,2024-01-01 09:00:00+00:00
1,track,Signup Completed,user_001,NaN,user_001,2024-01-01 09:02:00+00:00
2,track,Login,user_001,NaN,user_001,2024-01-02 08:30:00+00:00
3,track,Feature Used,user_001,NaN,user_001,2024-01-02 08:35:00+00:00
4,track,Feature Used,NaN,anon_456,anon_456,2024-01-02 10:00:00+00:00
5,identify,NaN,user_002,anon_456,user_002,2024-01-02 10:05:00+00:00
